In [ ]:
'''
dont forgot to use the version tags in git after changing the code . we have to use 
those v tags once we are done with entire code and making some minor updates 
we need to push those tags separately.

'''

In [ ]:
# GPT link : https://chatgpt.com/c/67c31846-7cec-8003-9bae-1f349c1f37c1

tensorflow → Main framework for deep learning.

🔹 keras → High-level API inside TensorFlow for building neural networks.

🔹 Sequential → A linear stack of layers for CNN.

🔹 Conv2D, MaxPooling2D, Flatten, Dense, Dropout → Different layers for building CNN.

🔹 ImageDataGenerator → Helps load and preprocess images.

🔹 numpy → Used for numerical operations.

🔹 matplotlib.pyplot → Helps plot training results.

🔹 cv2 → OpenCV for image processing (resizing, conversion).

🔹 os → Helps in handling file paths.

In [10]:
import numpy as np
import pandas as pd

import tensorflow as tf  
from tensorflow import keras  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

import matplotlib.pyplot as plt
import os


in "tensorflow.keras.preprocessing.image", each word separated by a dot (.) represents a package, module, or submodule in Python.
tensorflow → The main TensorFlow package.

keras → A submodule inside TensorFlow that provides deep learning APIs.

preprocessing → A submodule inside Keras that contains tools for data preprocessing.

image → A module inside preprocessing that specifically deals with image data (e.g., ImageDataGenerator, load_img, img_to_array).



REASON FOR TARGET SIZE
every image in dataset is 512x512

Why Does 512x512 Require More GPU?
More Pixels → More Computation

A 512x512 image has 262,144 pixels, whereas a 224x224 image has only 50,176 pixels.
That’s 5x more pixels, meaning 5x more computation per image!


In [11]:
# https://chatgpt.com/c/67d037a5-f280-8003-a91b-60bba6b2af07

# need to learn about mixed precision 

# Define ImageDataGenerator with Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,  
    rotation_range=40,       # Rotate images randomly by 40 degrees
    width_shift_range=0.2,   # Shift width randomly
    height_shift_range=0.2,  # Shift height randomly
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Zoom in/out
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest',     # Fill missing pixels
    validation_split=0.2     # 80-20 train-validation split
)

# ImageDataGenerator for Validation (No Augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,  
    validation_split=0.2
)




# Load Training Images with Augmentation
train_generator = train_datagen.flow_from_directory(
    "animals",  
    target_size=(224, 224),  
    batch_size=32,  
    class_mode="binary",  
    subset="training"
)

# Load Validation Images (No Augmentation)
validation_generator = val_datagen.flow_from_directory(
    "animals",  
    target_size=(224, 224),  
    batch_size=32,  
    class_mode="binary",  
    subset="validation"
)


Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [12]:
print(train_generator.class_indices)

{'cat': 0, 'dog': 1}


In [13]:
# https://chatgpt.com/c/67dc0951-d250-8003-af27-3128f21b2498

# Define the CNN model
model = Sequential([

    # 1st Convolutional Block
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),  # Normalize activations
    MaxPooling2D(2,2),

    # 2nd Convolutional Block
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # 3rd Convolutional Block
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Flatten and Fully Connected Layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(1, activation='sigmoid')  # Output Layer (Binary Classification)
])

# Print Model Summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_3 (Batc  (None, 222, 222, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_4 (Batc  (None, 109, 109, 64)     256       
 hNormalization)                                                 
                                                      

In [14]:
from tensorflow.keras.optimizers import Adam

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a small learning rate
    loss="binary_crossentropy",  # Loss function for binary classification
    metrics=["accuracy"]  # Metric to evaluate performance
)


In [31]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# change the folder for every trail

# Save a checkpoint every 2 epochs
model_checkpoint = ModelCheckpoint(
    r"checkpoints/first_trail/model_epoch_{epoch:02d}.h5",  # Saves model as "model_epoch_01.h5", "model_epoch_02.h5", etc.
    save_freq='epoch',  # Save after every epoch
    period=1  # Save every 2 epochs
)

# Early Stopping (optional, but useful)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Reduce learning rate if validation loss stops improving
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

# Combine callbacks
callbacks = [model_checkpoint, early_stopping, reduce_lr]


In [32]:
# Train the model
history = model.fit(
    train_generator,  
    validation_data=validation_generator,  
    epochs=30,  # Adjust based on performance
    callbacks=callbacks,  # Includes checkpointing, early stopping, etc.
    verbose=1  # Shows training progress
)


Epoch 1/30
25/25 [==============================] - 26s 1s/step - loss: 0.3310 - accuracy: 0.8625 - val_loss: 7.1429 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 2/30
25/25 [==============================] - 34s 1s/step - loss: 0.3426 - accuracy: 0.8612 - val_loss: 6.5370 - val_accuracy: 0.5050 - lr: 1.0000e-04
Epoch 3/30
25/25 [==============================] - 24s 964ms/step - loss: 0.2521 - accuracy: 0.8813 - val_loss: 5.4440 - val_accuracy: 0.5100 - lr: 1.0000e-04
Epoch 4/30
25/25 [==============================] - 25s 979ms/step - loss: 0.2699 - accuracy: 0.8950 - val_loss: 4.8150 - val_accuracy: 0.5150 - lr: 1.0000e-04
Epoch 5/30
25/25 [==============================] - 22s 894ms/step - loss: 0.2665 - accuracy: 0.8938 - val_loss: 3.7508 - val_accuracy: 0.5300 - lr: 1.0000e-04
Epoch 6/30
25/25 [==============================] - 25s 990ms/step - loss: 0.2232 - accuracy: 0.8975 - val_loss: 2.5965 - val_accuracy: 0.5750 - lr: 1.0000e-04
Epoch 7/30
25/25 [============================

In [33]:
# Evaluate on the validation set
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")


7/7 [==============================] - 3s 357ms/step - loss: 0.0441 - accuracy: 0.9750
Validation Accuracy: 97.50%
Validation Loss: 0.0441


In [2]:
from tensorflow.keras.models import load_model

# Path to your saved model (adjust the filename if needed)
model_path = r"C:\Users\abhir\Documents\project\dog vs cat\checkpoints\first_trail\model_epoch_30.h5"

# Load the trained model
model = load_model(model_path)

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Function to preprocess and predict an image
def predict_image(img_path, model):
    # Load image and resize it to match model input size
    img = image.load_img(img_path, target_size=(224, 224))  # Change target_size if needed

    # Convert image to array
    img_array = image.img_to_array(img)

    # Expand dimensions to match model input shape (batch of 1)
    img_array = np.expand_dims(img_array, axis=0)

    # Normalize the image (same as training)
    img_array /= 255.0  

    # Make prediction
    prediction = model.predict(img_array)[0][0]  # Get first value

    # Map prediction to class label
    if prediction > 0.5:
        label = "Dog 🐶"
    else:
        label = "Cat 🐱"

    print(f"Predicted class: {label} (Confidence: {prediction:.2f})")

# Test the function with your image



In [19]:
test_image_path = r"C:\Users\abhir\Downloads\cat4.jpeg"
predict_image(test_image_path, model)

1/1 [==============================] - 0s 29ms/step
Predicted class: Cat 🐱 (Confidence: 0.07)


In [ ]:
# 1/1 [==============================] - 0s 26ms/step
# Predicted class: Dog 🐶 (Confidence: 0.66

# the confidece is quite less as i have given an image which has two dogs instead of one , and the model haven't seen those till now an image with two dogs


# dog 3 and cat 3 doesn't worked well as they look quite different